In [38]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# 1. **Twitter Data Cleaning**
Downloaded a 2GB Kaggle dataset of BTC-related tweets. Timestamps span around 3 years at great granularity. Needed some cleaning as some observations where misplaced

In [2]:
twt = pd.read_csv('../Bitcoin_tweets.csv') # 2GB

/var/folders/xg/pf7prx2j6mngdfnzgpn5r9980000gn/T/ipykernel_65200/3423561035.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  twt = pd.read_csv('../Bitcoin_tweets.csv') # 2GB


In [8]:
ldates = twt['date'][twt['date'].apply(lambda x: str(x)[0] != '2')].dropna()
htags = ldates[ldates.str.startswith('[')] # observations with hashtags in date column
vrfy = ldates[~ldates.str.startswith('[')] # observations with tweet in date column

In [12]:
twt.loc[htags.index, 'text'] = twt.loc[htags.index, 'user_verified'] # put text in right place from user_verified
twt.loc[htags.index, 'date'] = twt.loc[htags.index, 'user_favourites'] # put dates in right place
twt.loc[vrfy.index, 'text'] = twt.loc[vrfy.index, 'date'] # put text in right place from date
twt.loc[vrfy.index, 'date'] = twt.loc[vrfy.index, 'user_verified'] # put date in right place

In [16]:
fudate = twt.loc[4524465].date # one last observation with text misplaced
twt.loc[4524465, 'text'] = fudate # put text in right place
twt.loc[4524465, 'date'] = twt.loc[4524465].user_verified # put date in right place

In [20]:
twt = twt.drop(2612133, axis=0) # drop an observation with wrong date and nan values

In [21]:
twt['date'] = pd.to_datetime(twt['date'], format="%Y-%m-%d %H:%M:%S")

In [40]:
tweets = twt.loc[:, ['date', 'text']].copy()
tweets.set_index('date', inplace=True)

# 2. **Regulatory Dates**

In [28]:
regtime = pd.read_csv('./crypto_reg_timeline.csv')
regtime['Date'] = pd.to_datetime(regtime['Date'].astype(str))
regtime.rename({'Date': 'date'}, axis=1, inplace=True)
regtime.head()

,date,Country/Region,Topic
0,2018-06-11,UK,AML/CFT
1,2019-10-01,US,Cyrpto firms can become SPDIs
2,2019-12-16,HK,Risk assessment for VASPs and AML/CFT
3,2020-01-30,US,OCC issued cease-and-desist to Safra Bank for ...
4,2020-06-04,US,OCC fair access to banking services


In [116]:
regtime

,date,Country/Region,Topic
0,2018-06-11,UK,AML/CFT
1,2019-10-01,US,Cyrpto firms can become SPDIs
2,2019-12-16,HK,Risk assessment for VASPs and AML/CFT
3,2020-01-30,US,OCC issued cease-and-desist to Safra Bank for ...
4,2020-06-04,US,OCC fair access to banking services
5,2020-06-22,US,OCC banks can provide crypto custodiary services
6,2020-09-16,US,Kraken becomes SPDI
7,2020-09-21,US,OCC banks can hold reserves for stablecoin iss...
8,2021-01-04,US,OCC banks can engage in some payment-activitie...
9,2021-01-11,US,OCC recognizes authority of crypto business as...


In [129]:
bcbs_date = pd.Timestamp('2021-02-05')
dt = pd.Timedelta(days=20)

In [127]:
regtime.loc[regtime.date == bcbs_date].Topic.iloc[0]

'BCBS final standard on prudential treatment for crypto exposures'

# 3. **Sentiment Analysis**

In [41]:
#sentiment analysis for postive or negative
nltk.download('vader_lexicon')

def get_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']
    
tweets['sentiment_score'] = tweets['text'].apply(get_sentiment)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/MicheleOrlandi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


KeyboardInterrupt: 